# ソース付き検索質問応答

このノートブックでは、インデックス上のソースを使用して質問応答を行う方法について説明します。これは、`RetrievalQAWithSourcesChain`を使用してインデックスからドキュメントを検索することで実現します。

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma

In [ ]:
with open("../../state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

In [ ]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": f"{i}-pl"} for i in range(len(texts))])

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain

In [ ]:
chain = RetrievalQAWithSourcesChain.from_chain_type(ChatOpenAI(temperature=0, model_name="gpt-4"), chain_type="stuff", retriever=docsearch.as_retriever())

In [ ]:
chain({"question": "ブレイヤー判事について、大統領はどのように語ったか"}, return_only_outputs=True)

## チェーンタイプ
RetrievalQAWithSourcesChainチェーンで、異なるチェーンタイプを簡単に指定してロードして使用できます。これらのタイプの詳細な解説については、[このノートブック](qa_with_sources.ipynb)を参照してください。

異なるチェーンタイプをロードする方法は2つあります。まず、`from_chain_type`メソッドでチェーンタイプ引数を指定できます。これにより、使用するチェーンタイプの名前を渡すことができます。例えば、以下ではチェーンタイプを`map_reduce`に変更しています。

In [ ]:
chain = RetrievalQAWithSourcesChain.from_chain_type(ChatOpenAI(temperature=0,model_name="gpt-4"), chain_type="map_reduce", retriever=docsearch.as_retriever())

In [ ]:
chain({"question": "What did the president say about Justice Breyer"}, return_only_outputs=True)

上記の方法では、chain_typeを非常に簡単に変更できますが、そのchain_typeに対するパラメータの柔軟性はたくさん提供されています。これらのパラメータを制御したい場合は、チェーンを直接ロードして（[このノートブック](qa_with_sources.ipynb)で行ったように）、`combine_documents_chain`パラメータを使って、そのチェーンをRetrievalQAWithSourcesChainチェーンに直接渡すことができます。例えば：

In [ ]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
qa_chain = load_qa_with_sources_chain(ChatOpenAI(temperature=0), chain_type="stuff")
qa = RetrievalQAWithSourcesChain(combine_documents_chain=qa_chain, retriever=docsearch.as_retriever())

In [ ]:
qa({"question": "What did the president say about Justice Breyer"}, return_only_outputs=True)